In [1]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 4.2 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split, KFold, GridSearchCV, cross_val_score
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR

In [3]:
pd.set_option('display.max_columns', None)

In [5]:
df = pd.read_csv('dataset_collaboration_with_survey_scores.csv')

df.head()

,id,project,meeting_number,speaker_number,speech_frequency,total_words,duration,normalized_speech_frequency,speaker_id,next_speaker_id,count,network_density,weighted_network_density,gini_coefficient,interaction_equality_index,degree_centrality,indegree_centrality,outdegree_centrality,betweenness_centrality,closeness_centrality,eigenvector_centrality,pagerank,overall_collaboration_score,individual_collaboration_score
0,3_1_SPEAKER_00,3,1,0,22,8731,1.633333,13.469388,0,0,0,0.583333,0.129211,0.518229,0.009704,0.000094,0.000047,0.000047,0.000000,9.518792e-08,0.0,0.000003,-1.0,-1.0
1,3_1_SPEAKER_00,3,1,0,22,8731,1.633333,13.469388,0,1,1,0.583333,0.129211,0.518229,0.009704,0.000094,0.000047,0.000047,0.000000,9.518792e-08,0.0,0.000003,-1.0,-1.0
2,3_1_SPEAKER_00,3,1,0,22,8731,1.633333,13.469388,0,2,0,0.583333,0.129211,0.518229,0.009704,0.000094,0.000047,0.000047,0.000000,9.518792e-08,0.0,0.000003,-1.0,-1.0
3,3_1_SPEAKER_00,3,1,0,22,8731,1.633333,13.469388,0,3,0,0.583333,0.129211,0.518229,0.009704,0.000094,0.000047,0.000047,0.000000,9.518792e-08,0.0,0.000003,-1.0,-1.0
4,3_1_SPEAKER_01,3,1,1,645,8731,1.633333,394.897959,1,0,1,0.583333,0.129211,0.518229,0.009704,0.166808,0.083381,0.083428,0.000051,9.523587e-08,0.0,0.000030,-1.0,-1.0


전처리

In [6]:
df = df[(df['project'] == 4) & (df['individual_collaboration_score'].between(1, 10))]
df = df[df['speaker_id'] != df['next_speaker_id']]

df.head()

,id,project,meeting_number,speaker_number,speech_frequency,total_words,duration,normalized_speech_frequency,speaker_id,next_speaker_id,count,network_density,weighted_network_density,gini_coefficient,interaction_equality_index,degree_centrality,indegree_centrality,outdegree_centrality,betweenness_centrality,closeness_centrality,eigenvector_centrality,pagerank,overall_collaboration_score,individual_collaboration_score
81,4_1_SPEAKER_00,4,1,0,1294,12023,1.983333,652.436975,0,1,1,0.8,0.131715,0.401351,0.147041,0.098437,0.047005,0.051432,0.000022,9.955082e-07,0.000029,0.000013,3.0,3.0
82,4_1_SPEAKER_00,4,1,0,1294,12023,1.983333,652.436975,0,2,1,0.8,0.131715,0.401351,0.147041,0.098437,0.047005,0.051432,0.000022,9.955082e-07,0.000029,0.000013,3.0,4.0
83,4_1_SPEAKER_00,4,1,0,1294,12023,1.983333,652.436975,0,3,48,0.8,0.131715,0.401351,0.147041,0.098437,0.047005,0.051432,0.000022,9.955082e-07,0.000029,0.000013,3.0,3.0
84,4_1_SPEAKER_00,4,1,0,1294,12023,1.983333,652.436975,0,4,8,0.8,0.131715,0.401351,0.147041,0.098437,0.047005,0.051432,0.000022,9.955082e-07,0.000029,0.000013,3.0,4.0
85,4_1_SPEAKER_01,4,1,1,711,12023,1.983333,358.487395,1,0,3,0.8,0.131715,0.401351,0.147041,0.013715,0.006489,0.007227,0.000011,1.094421e-06,0.000004,0.000003,2.0,3.0


In [7]:
df['num_speakers'] = df.groupby(['project', 'meeting_number'])['speaker_number'].transform('nunique')

In [8]:
df['interaction_count'] = df['count']
if 'interaction_count' not in df.columns:
    df['interaction_count'] = 0

df['normalized_interaction_frequency'] = df['interaction_count'] / df['duration']

In [9]:
df = df.replace([np.inf, -np.inf], np.nan)
df = df.dropna()

In [10]:
df.head()

,id,project,meeting_number,speaker_number,speech_frequency,total_words,duration,normalized_speech_frequency,speaker_id,next_speaker_id,count,network_density,weighted_network_density,gini_coefficient,interaction_equality_index,degree_centrality,indegree_centrality,outdegree_centrality,betweenness_centrality,closeness_centrality,eigenvector_centrality,pagerank,overall_collaboration_score,individual_collaboration_score,num_speakers,interaction_count,normalized_interaction_frequency
81,4_1_SPEAKER_00,4,1,0,1294,12023,1.983333,652.436975,0,1,1,0.8,0.131715,0.401351,0.147041,0.098437,0.047005,0.051432,0.000022,9.955082e-07,0.000029,0.000013,3.0,3.0,5,1,0.504202
82,4_1_SPEAKER_00,4,1,0,1294,12023,1.983333,652.436975,0,2,1,0.8,0.131715,0.401351,0.147041,0.098437,0.047005,0.051432,0.000022,9.955082e-07,0.000029,0.000013,3.0,4.0,5,1,0.504202
83,4_1_SPEAKER_00,4,1,0,1294,12023,1.983333,652.436975,0,3,48,0.8,0.131715,0.401351,0.147041,0.098437,0.047005,0.051432,0.000022,9.955082e-07,0.000029,0.000013,3.0,3.0,5,48,24.201681
84,4_1_SPEAKER_00,4,1,0,1294,12023,1.983333,652.436975,0,4,8,0.8,0.131715,0.401351,0.147041,0.098437,0.047005,0.051432,0.000022,9.955082e-07,0.000029,0.000013,3.0,4.0,5,8,4.033613
85,4_1_SPEAKER_01,4,1,1,711,12023,1.983333,358.487395,1,0,3,0.8,0.131715,0.401351,0.147041,0.013715,0.006489,0.007227,0.000011,1.094421e-06,0.000004,0.000003,2.0,3.0,5,3,1.512605


In [11]:
df.shape

(340, 27)

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 340 entries, 81 to 525
Data columns (total 27 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   id                                340 non-null    object 
 1   project                           340 non-null    int64  
 2   meeting_number                    340 non-null    int64  
 3   speaker_number                    340 non-null    int64  
 4   speech_frequency                  340 non-null    int64  
 5   total_words                       340 non-null    int64  
 6   duration                          340 non-null    float64
 7   normalized_speech_frequency       340 non-null    float64
 8   speaker_id                        340 non-null    int64  
 9   next_speaker_id                   340 non-null    int64  
 10  count                             340 non-null    int64  
 11  network_density                   340 non-null    float64
 12  weighted_net

individual score

In [13]:
boost_models = {
    'Linear Regression': LinearRegression(),
    'Decision Tree': DecisionTreeRegressor(random_state=42),
    'Random Forest Regressor': RandomForestRegressor(random_state=42),
    'XGBRegressor': XGBRegressor(random_state=42),
    'Gradient Boosting Regressor': GradientBoostingRegressor(random_state=42),
    'K-Nearest Neighbors Regressor': KNeighborsRegressor(),
    'LightGBM Regressor': LGBMRegressor(random_state=42, verbose=-1),
    'CatBoost Regressor': CatBoostRegressor(random_state=42, verbose=0),
    'SVM Regressor': SVR()
}

In [14]:
param_grids_regression = {
    'Linear Regression': {},
    'Decision Tree': {'max_depth': [3, 5, 7]},
    'Random Forest Regressor': {'n_estimators': [50, 100, 150], 'max_depth': [5, 10, 15]},
    'XGBRegressor': {'n_estimators': [50, 100], 'max_depth': [3, 5, 7], 'learning_rate': [0.01, 0.1, 0.2]},
    'Gradient Boosting Regressor': {'n_estimators': [50, 100], 'max_depth': [3, 5], 'learning_rate': [0.01, 0.1, 0.2, 0.5]},
    'K-Nearest Neighbors Regressor': {'n_neighbors': [3, 5, 7]},
    'LightGBM Regressor': {'n_estimators': [50, 100, 200], 'num_leaves': [31, 62], 'learning_rate': [0.01, 0.1, 0.3]},
    'CatBoost Regressor': {'iterations': [100, 200, 400], 'depth': [4, 6, 10]},
    'SVM Regressor': {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']}
}

In [16]:
# Initialize dictionaries to store scores and feature importances
mse_scores = {model_name: [] for model_name in boost_models}
mae_scores = {model_name: [] for model_name in boost_models}
r2_scores = {model_name: [] for model_name in boost_models}
feature_importances = {model_name: [] for model_name in boost_models}
execution_times = {model_name: [] for model_name in boost_models}
best_params = {}

# Features and target
X = df[['meeting_number', 'normalized_speech_frequency', 'gini_coefficient', 'degree_centrality', 'num_speakers', 'normalized_interaction_frequency', 'speaker_id', 'next_speaker_id']]
y = df['individual_collaboration_score']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# One-Hot Encoding for categorical variables
categorical_features = ['speaker_id', 'next_speaker_id']
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(), categorical_features),
        ('num', StandardScaler(), X.drop(columns=categorical_features).columns)
    ])

# KFold initialization
kf = KFold(n_splits=10, shuffle=True, random_state=42)

for model_name, model in boost_models.items():
    param_grid = param_grids_regression[model_name]

    # Create a pipeline that first transforms the data and then applies the model
    pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('model', model)])

    # GridSearchCV with pipeline
    grid_search = GridSearchCV(estimator=pipeline, param_grid={'model__' + key: value for key, value in param_grid.items()}, scoring='r2', cv=3, verbose=0)

    # Measure start time
    start_time = time.time()

    # Fit grid search
    grid_search.fit(X_train, y_train)

    # Measure end time
    end_time = time.time()
    execution_times[model_name] = round(end_time - start_time, 2)

    # Get best model and predict
    best_model = grid_search.best_estimator_
    y_pred = best_model.predict(X_test)

    # Save the best parameters
    best_params[model_name] = grid_search.best_params_

    # Calculate metrics
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    # Append scores to dictionaries
    mse_scores[model_name].append(round(mse, 2))
    mae_scores[model_name].append(round(mae, 2))
    r2_scores[model_name].append(round(r2, 2))

    # Cross-validation scores
    cv_scores = cross_val_score(best_model, X_train, y_train, cv=kf, scoring='r2')
    mean_cv_score = np.mean(cv_scores)
    std_cv_score = np.std(cv_scores)

    # Append performance
    print(f'[{model_name}] R-squared Score: {round(r2, 2)}')
    print(f'[{model_name}] Mean Squared Error: {round(mse, 2)}')
    print(f'[{model_name}] Mean CV Score: {round(mean_cv_score, 2)}')
    print(f'[{model_name}] Std CV Score: {round(std_cv_score, 2)}')
    print(f'[{model_name}] Execution Time: {execution_times[model_name]} seconds')
    print(f'[{model_name}] Best Parameters: {best_params[model_name]}')
    print('------------------------------------------------------------------\n')

    # Check if the model has feature importances attribute
    if hasattr(best_model.named_steps['model'], 'feature_importances_'):
        feature_importances[model_name].append(best_model.named_steps['model'].feature_importances_)

# Print best model information
best_model_name = max(r2_scores, key=lambda k: r2_scores[k][0])

print(f'Best Model: {best_model_name}')
print(f'Mean Squared Error: {mse_scores[best_model_name][0]}')
print(f'R-squared Score: {r2_scores[best_model_name][0]}')
print(f'Mean CV Score: {round(np.mean(cv_scores), 2)}')
print(f'Std CV Score: {round(np.std(cv_scores), 2)}')
print(f'Execution Time: {execution_times[best_model_name]} seconds')
print(f'Best Parameters: {best_params[best_model_name]}')

[Linear Regression] R-squared Score: 0.73
[Linear Regression] Mean Squared Error: 1.2
[Linear Regression] Mean CV Score: 0.65
[Linear Regression] Std CV Score: 0.2
[Linear Regression] Execution Time: 0.07 seconds
[Linear Regression] Best Parameters: {}
------------------------------------------------------------------

[Decision Tree] R-squared Score: 0.74
[Decision Tree] Mean Squared Error: 1.16
[Decision Tree] Mean CV Score: 0.62
[Decision Tree] Std CV Score: 0.15
[Decision Tree] Execution Time: 0.2 seconds
[Decision Tree] Best Parameters: {'model__max_depth': 3}
------------------------------------------------------------------

[Random Forest Regressor] R-squared Score: 0.81
[Random Forest Regressor] Mean Squared Error: 0.84
[Random Forest Regressor] Mean CV Score: 0.76
[Random Forest Regressor] Std CV Score: 0.09
[Random Forest Regressor] Execution Time: 5.01 seconds
[Random Forest Regressor] Best Parameters: {'model__max_depth': 10, 'model__n_estimators': 150}
-------------------